In [2]:
!pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   --------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# ETL Pipeline: Data Extraction & Transformation

## Step 1: Install Dependencies
Installing required packages for data processing and Excel file handling.

## Step 2: Load & Explore Data
Loading the Excel dataset and examining its structure to understand the data format, dimensions, and initial content.

In [10]:

#  1_extract_transform.ipynb
# ETL: Data Cleaning & Enrichment

# Import required libraries
import pandas as pd
import openpyxl
import numpy as np


#  EXTRACT

# Path to dataset
file_path = r"C:\Users\Admin\Desktop\data\Group work dsa.xlsx"

# Load the Excel dataset
df = pd.read_excel(file_path)

# Preview first few rows
print("Original Data Preview:")
print(df.head())
print(f"Initial row count: {len(df)}")

# Check dataset info
print("\nDataset Info:")
df.info()


Original Data Preview:
   location_id location_code location_name  year  age_group_id age_group_name  \
0            1             G        Global  1990             1        Under 5   
1            1             G        Global  1990             1        Under 5   
2            1             G        Global  1990             1        Under 5   
3            1             G        Global  1990             1        Under 5   
4            1             G        Global  1990             1        Under 5   

   sex_id    sex_name  cause_id    cause_name      metric    unit  \
0       1       Males       297  Tuberculosis      Deaths  Number   
1       1       Males       297  Tuberculosis  Prevalence  Number   
2       2     Females       297  Tuberculosis      Deaths  Number   
3       2     Females       297  Tuberculosis  Prevalence  Number   
4       3  Both sexes       297  Tuberculosis      Deaths  Number   

          mean    lower     upper  
0  42874.89844  34608.3   63349.5  
1  

## Step 3: Check Data Quality
Identifying missing values in each column to understand the extent of missing data before cleaning.

In [ ]:
# check for missing values
print("\nMissing Values in Each Column:")
print(df.isnull().sum())



Missing Values in Each Column:
location_id       0
location_code     0
location_name     0
year              0
age_group_id      0
age_group_name    0
sex_id            0
sex_name          0
cause_id          0
cause_name        0
metric            0
unit              0
mean              0
lower             0
upper             0
dtype: int64


## Step 4: Data Cleaning & Enrichment
Filtering for African countries, removing old data (1990-1999), dropping unwanted columns, handling missing values, removing outliers and noise, adding calculated fields, and standardizing data types. Finally, saving the cleaned dataset to a new Excel file.

In [9]:



# TRANSFORM: Enhanced Clean & Enrich

#  Keep only African countries + Global
african_countries = [
    "Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi",
    "Cabo Verde", "Cameroon", "Central African Republic", "Chad", "Comoros",
    "Congo", "Democratic Republic of the Congo", "Djibouti", "Egypt",
    "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia",
    "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho",
    "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania",
    "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria",
    "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone",
    "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo",
    "Tunisia", "Uganda", "Zambia", "Zimbabwe", "Global"
]

if 'location_name' in df.columns:
    df['location_name'] = df['location_name'].str.strip().str.title()
    df = df[df['location_name'].isin(african_countries)]
    print(f" Kept African countries + Global. Remaining rows: {len(df)}")
else:
    print(" Column 'location_name' not found — check dataset.")

#  Remove years 1990–1999
if 'year' in df.columns:
    df = df[~df['year'].between(1990, 1999)]
    print(f" Removed years 1990–1999. Remaining rows: {len(df)}")
else:
    print(" Column 'year' not found — check dataset.")
    print(f" Remaining rows: {len(df)}")


#Remove unwanted columns: location_code, cause_id
cols_to_drop = ['location_code', 'cause_id']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
print(f" Dropped columns: {cols_to_drop}")

# Handle missing numeric values
numeric_cols = ['mean', 'lower', 'upper']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())
        print(f" Filled missing values in '{col}' with mean: {df[col].mean()}")

# Handle missing categorical values
categorical_cols = ['age_group_name', 'sex_name', 'cause_name', 'metric', 'unit']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])
        df[col] = df[col].str.strip() if df[col].dtype == 'object' else df[col]

# Outlier Handling: Remove extreme values in numeric columns
for col in numeric_cols:
    if col in df.columns:
        lower, upper = df[col].quantile([0.01, 0.99])
        df = df[(df[col] >= lower) & (df[col] <= upper)]
print(" Outliers removed using 1st and 99th percentiles.")
print(f" Remaining rows after outlier removal: {len(df)}")

# Noise removal: Remove negative or impossible values in 'mean', 'lower', 'upper'
for col in numeric_cols:
    if col in df.columns:
        df = df[df[col] >= 0]
print(" Noise removed: negative values cleaned.")
print(f" Remaining rows after cleaning: {len(df)}")

# Calculated field: % deviation of upper from mean
if all(col in df.columns for col in ['mean', 'upper']):
    df['upper_deviation_pct'] = ((df['upper'] - df['mean']) / df['mean']) * 100
    print(" Added calculated field 'upper_deviation_pct'.")
    print(df[['mean', 'upper', 'upper_deviation_pct']].head())
    print(f" Remaining rows: {len(df)}")

# Standardize year as integer
if 'year' in df.columns:
    df['year'] = df['year'].astype(int)

# Final preview
print("\nTransformed Data Preview:")
print(df.head())
print(f" Final row count: {len(df)}")


# Save cleaned dataset
cleaned_path = r"C:\Users\Admin\Desktop\data\Group_work_cleaned.xlsx"
df.to_excel(cleaned_path, index=False)
print(f"\n Cleaned dataset saved at: {cleaned_path}")



 Kept African countries + Global. Remaining rows: 83307
 Removed years 1990–1999. Remaining rows: 83307
 Dropped columns: ['location_code', 'cause_id']
 Filled missing values in 'mean' with mean: 468.28346843565134
 Filled missing values in 'lower' with mean: 322.17529979473517
 Filled missing values in 'upper' with mean: 658.8869758843795
 Outliers removed using 1st and 99th percentiles.
 Remaining rows after outlier removal: 78651
 Noise removed: negative values cleaned.
 Remaining rows after cleaning: 78651
 Added calculated field 'upper_deviation_pct'.
           mean  upper  upper_deviation_pct
792   41.000000   45.5            10.975610
793  260.299988  268.6             3.188633
794   22.900000   24.9             8.733626
795  123.099998  126.9             3.086922
796   31.200001   33.5             7.371792
 Remaining rows: 78651

Transformed Data Preview:
     location_id location_name  year  age_group_id    age_group_name  sex_id  \
792            1        Global  2000       